<h1 align="center">Machine Learning</h1><h2 align="center" style="margin:10px">Assignment 3</h2>

Group 6:
    287040 Stephan Thierry
    254172 Kasper Holst Daugaard 


The assignments below should be solved and documented as a mini-project that will form the basis for the
examination. When solving the exercises it is important that you

  * document all relevant results and analyses that you have obtained/performed during the exercises
  * try to relate your results to the theoretical background of the methods being applied.

Feel free to add cells if you need to. The easiest way to convert to pdf is to save this notebook as .html (File-->Download as-->HTML) and then convert this html file to pdf. You can also export as pdf directly, but here you need to watch your margins as the converter will cut off your code (i.e. make vertical code!).

In [ ]:
# Import all necessary libraries here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import mglearn

# Exercise 1: Linear vs nonlinear classifiers

Below, we show a dataset that cannot be linearly separated. In this exercise, we will use the default parameters for all classifiers (except the custom SVM in exercise d).

In [ ]:
from sklearn.datasets.samples_generator import make_circles
X, y = make_circles(1000, factor=0.0, noise=0.2)



a) Plot the dataset e.g. using the `discrete_scatter`-function from mglearn.

In [ ]:
mglearn.discrete_scatter(X[:,0],X[:,1], y)


b) Split the dataset into train and test-sets.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

Here we could split using a random_state that demonstrates the outcome that we want to show (3). Other random_state's (4) will show a much lower accuracy - but knowing that data it should be about 50/50.

Instead we stratify on y so the data is split evenly and there is not an overrepresentation of one classification in either of the new datasets

We don't set "test_size" so we use the default split of 75/25.

c) Train a logistic regression on the dataset, and compute the classification accuracy. 

In [ ]:
from sklearn.linear_model import LogisticRegression
Logreg = LogisticRegression(solver='lbfgs')
##Logreg.fit(X_train,y_train)
Logreg.fit(X_train,y_train)
print (Logreg.predict_proba([X_test[0]]))
print (Logreg.predict_proba([X_test[10]]))
print (Logreg.predict_proba([X_test[100]]))

print("Accuracy on Training set is: {:.2f}".format(Logreg.score(X_train,y_train)))
print("Accuracy on Test set is: {:.2f}".format(Logreg.score(X_test,y_test)))

We train the model on the training set at test the accuracy on the test set. As expected it's around 50%.

d) Plot the decision boundary for the logistic regression (e.g. using the `plot_2d_separator`-function from mglearn), and use this to investigate why the algorithm does not give a good result in this case.

In [ ]:
Logreg.fit(X_train,y_train)
from mglearn.plots import plot_2d_separator
mglearn.discrete_scatter(X_train[:,0],X_train[:,1], y_train)
plt.plot(X_train[0,0],X_train[0,1],'k.')
plt.legend()
plot_2d_separator(Logreg, X_train, fill=True, eps=0.4, alpha=.7)




We can see the linier boundry in a 2d space will not accurately seperate the data. It's split down the middle so ~50/50 is the best we can hope for.

e) Think of a feature you could add to this dataset to make it linearly separable. 
Add this feature, retrain the logistic regression classifier, and compute the accuracy again. Comment on the result.

Since the data is a circle inside another circle, adding "distance to the center" as a feature would be useful to the liner (plane) seperation 

From using distance from scipy.spatial we calculate the euclidean distance to the center. The center is at 0,0 - but if it was not we could calculate is and change the center-variable accordingly

See code comments for details:

In [ ]:
## Add distance as new feature

from scipy.spatial import distance
from sklearn.svm import LinearSVC

center = [0,0];
X2 = np.empty([0,3])

#We run through each row of X and add the distance as a feature in each row
for eachitem in X:
    # Calculate distance to center
    dist = distance.euclidean(center, eachitem)
    # Add (insert) the distance to the current row and add (vertical-stack) the result to the resultset: X2 
    X2 = np.vstack((X2,np.insert(eachitem, 2, dist)))

## print(X2)

# We do a new split so test and traing sets contain the new feature
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, stratify=y)

## We use the X_plot and y_plot vaiables so we can run the plot-code on both 
##  the full dataset and the training-set by only changing 2 lines

##X_plot = X2
##y_plot = y

X_plot = X2_train
y_plot = y2_train

In [ ]:

from mpl_toolkits.mplot3d import Axes3D, axes3d
figure = plt.figure()
# visualize in 3D
ax = Axes3D(figure, elev=-152, azim=-26)
# plot first all the points with y == 0, then all with y == 1
mask = y_plot == 0
ax.scatter(X_plot[mask, 0], X_plot[mask, 1], X_plot[mask, 2], c='b',
    cmap=mglearn.cm2, s=60)
ax.scatter(X_plot[~mask, 0], X_plot[~mask, 1], X_plot[~mask, 2], c='r', marker='^',
    cmap=mglearn.cm2, s=60)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Distance to center")

linear_3d = Logreg.fit(X_plot, y_plot)
coef, intercept = linear_3d.coef_.ravel(), linear_3d.intercept_
# show linear decision boundary
figure = plt.figure()
ax = Axes3D(figure, elev=-152, azim=-26)
xx = np.linspace(X_plot[:, 0].min() - 2, X_plot[:, 0].max() + 2, 50)
yy = np.linspace(X_plot[:, 1].min() - 2, X_plot[:, 1].max() + 2, 50)
XX, YY = np.meshgrid(xx, yy)
ZZ = (coef[0] * XX + coef[1] * YY + intercept) / -coef[2]
ax.plot_surface(XX, YY, ZZ, rstride=8, cstride=8, alpha=0.3)
ax.scatter(X_plot[mask, 0], X_plot[mask, 1], X_plot[mask, 2], c='b',
    cmap=mglearn.cm2, s=60)
ax.scatter(X_plot[~mask, 0], X_plot[~mask, 1], X_plot[~mask, 2], c='r', marker='^',
    cmap=mglearn.cm2, s=60)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Distance to center")

print('Accuracy on training set: {:.2f}'.format(linear_3d.score(X2_train, y2_train)))
print('Accuracy on test set: {:.2f}'.format(linear_3d.score(X2_test, y2_test)))


f) Now, return to the original dataset (without the extra feature), and train a kernelized SVM on the dataset. Compute the accuracy and plot the decision boundary. Compare to your previous results and discuss the differences.

In [ ]:
from sklearn.svm import SVC
svm = SVC(gamma='auto').fit(X_train,y_train)

mglearn.discrete_scatter(X[:, 0], X[:, 1], y)

##fig, axes = plt.subplots(figsize=(10, 3))

plot_2d_separator(svm, X, fill=True, eps=0.5, alpha=.7 )
##plt.figure(figsize=(20,20))
print("Accuracy on training set: {:.2f}".format(svm.score(X_train, y_train)))
print("Accuracy on test set: {:.2f}".format(svm.score(X_test, y_test)))
plt.xlabel("X")
plt.ylabel("Y")



The kernelized SVM does not rely on a liniear seperation of the data, so we don't need to move into a higher dimintion to achive accurate seperation.



In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(25,20))

for ax, C in zip(axes, [0.1, 1, 1000]):
    for a, gamma in zip(ax,[0.1, 1, 10]):
        svm = SVC(C=C, gamma=gamma).fit(X,y)
        mglearn.discrete_scatter(X[:,0],X[:,1], y, ax=a)
        mglearn.plots.plot_2d_separator(svm, X, eps=.5, fill=True, alpha=0.3, ax=a)
        a.set_title("C={:.3f}, gamma={:.3f}".format(C,gamma))
        
svm.fit(X_train, y_train)
print("Accuracy on training set: {:.2f}".format(svm.score(X_train, y_train)))
print("Accuracy on test set: {:.2f}".format(svm.score(X_test, y_test)))

# Exercise 2: MNIST

In [ ]:
print(axes);

In this exercise, we consider the famous MNIST dataset, which is loaded below. The part of the dataset loaded as `testX` and `testY` will be reserved for testing - i.e. these cannot be used at all during training. 

It might be a good idea to only use part of the dataset (`X` and `Y`) while tuning parameters (in order to reduce the training-time).

In [ ]:
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data()

The code-snippet below can be used to see the digits corresponding to individual digits:

In [ ]:
import matplotlib.pyplot as plt
index = 1

plt.imshow(X[index].reshape(28,28),cmap=plt.cm.gray_r)
plt.show()

a) Split the training data into a training and a validation set.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, Y, stratify=Y)

### Binary classification

b) To begin with, in order to make things a little bit simpler (and faster!), extract from the data a binary subset, that only contains the data for two selected digits:

In [ ]:
import numpy as np

## Take two digits 5 and 2

reducedIndexes = np.isin(y, [2,5])
X_subset, y_subset = X[reducedIndexes], y[reducedIndexes]


X_train, X_validation, y_train, y_validation = train_test_split(X_subset, y_subset, stratify=Y)

print(y_train.size) # 41250 elements
print(y_train_subset.size) # reduced to 7843 elements (if we dont use the "break")

plt.imshow(X_train_subset[0].reshape(28,28),cmap=plt.cm.gray_r)
plt.show()


c) Learn different SVM models by varying the kernel function. For each configuration,
determine the time it takes to learn the model, and the accuracy on the validation data. *Caution*: for some
configurations, learning here can take a little while (several minutes).

In [ ]:
#It must be one of ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’
X_train, X_test, y_train, y_test = train_test_split(X_train_subset, y_train_subset, stratify=y_train_subset)

import mglearn
import time
numberList = [1, 2, 3]
kernelList = [1, 2, 3, 4]

for C in [0.1, 1, 1000]:
    for gamma in [0.1, 1, 10]:
        for kernel in ['poly','rbf','sigmoid']:
            start = time.time()
            svm = SVC(C=C, gamma=gamma, kernel=kernel).fit(X_train,y_train)
            end = time.time()
            #mglearn.discrete_scatter(X_train_subset[:,0],X_train_subset[:,1], y_train_subset, ax=a)
            #mglearn.plots.plot_2d_separator(svm, X_train_subset, eps=.5, fill=True, alpha=0.3, ax=a)
            #a.set_title("C={:.3f}, gamma={:.3f}".format(C,1))
            print("C: " + str(C) + " gamma: " + str(gamma) + " Kernel: "+ kernel + " Time: " + str(end - start))
            print("Accuracy on training subset: {:.2f}".format(svm.score(X_train, y_train)))
            print("Accuracy on test subset: {:.2f}".format(svm.score(X_test, y_test)))
            print("")

d) Find a way to extract the misclassified test cases. Inspect some misclassified cases and display them along with their correct label.
Do they correspond to hard to recognize digits (also for the human reader)?  

In [ ]:
svm = SVC(C=1, gamma=1, kernel='poly').fit(X_train,y_train)
print("Accuracy on training subset: {:.2f}".format(svm.score(X_train, y_train)))
print("Accuracy on test subset: {:.2f}".format(svm.score(X_test, y_test)))
import matplotlib.pyplot as plt


misclassified = np.where(y_test != svm.predict(X_test))
misclassified = misclassified[0]
predictions = svm.predict(X_test)
for i in range(8):
    print("Prediction: "  + str(predictions[misclassified[i]]))
    print("Actual: " + str(y_test[misclassified[i]]))
    
    plt.imshow(X_test[misclassified[i]].reshape(28,28),cmap=plt.cm.gray_r)
    plt.show()


e) How do results (time and accuracy) change, depending on whether you consider
an 'easy' binary task (e.g. distinguishing '1' and '0') or a more difficult one (e.g. '4' vs. '5'). This exercise
requires you to make new datasets with different values for 'digit1' and 'digit2'.

### Multiclass classification

f) [Discussion only] Explain how a binary classifier, such as an SVM, can be applied to a multiclass classification problem, such as recognizing all 10 digits in the MNIST dataset.

g) From the binary classification exercise above, identify a good configuration that gives a reasonable combination
of accuracy and runtime. Use this configuration to perform a full classification of the 10 classes in the
original dataset. Report the accuracy obtained on the test data.

# Exercise 3: Regression with random forest

For this exercise we will be using the famous nycflights dataset.

So far, we have only considered how to use SVMs and decision trees (and, by extension, random forests) for classification. However, both algorithms can also be used for regression tasks, as we will see in the exercises below.

### Preprocessing

a) Load the data as a pandas dataframe and display the first 5 rows of the dataset. Remove the columns `'carrier'`,`'tailnum'`,`'flight'`,`'origin'`, and `'dest'`.

In [ ]:
import pandas as pd

flights= pd.read_csv('flights.csv')
for col in ['carrier','tailnum','flight','origin','dest']:
    flights = flights.drop(columns=col)
print("Columns after drop: " + str(flights.columns))

# Display first 5 rows
print(flights[:5])

b) Plot the distributions for all variables (hint: use the `hist` method for the dataframe). Consider if you want to transform any of the variables, i.e. using a logarithmic transformation. Explain your choices.

Unlike standard regression methods, decition trees do not rely on calculating coefficients so the requirements for data preparation are not the same as for SVM or other regression-type functions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
flights.hist( figsize=(14,16))


c) Handle any nan-values in the dataset, and normalize all relevant variables. Are there any categorical variables? If so, create dummy variables for these.

In [ ]:
## This section is to detect 

print("Null values exist in the dataset (true or false) " + str(flights.isnull().any().any()))
print()
print("In what features can we find null values: ")
print(flights.isnull().any())
print()
print("How many?")
print(flights.isnull().sum())


In [ ]:
# We have the option to delete entries with Null or to replace them with another value 
#  like the mean, the mode or the median - in this case we use the 
#  'mode' meaning the most frequent observation. The code for "mean" is included in the comment

for col in ['dep_time','dep_delay','arr_time','arr_delay','air_time','hour','minute']:
    flights[col] = flights[col].fillna(flights[col].mode()[0])
    #flights[col] = flights[col].fillna(flights[col].mean())

# We now see that we have 0 nulls in our dataset
print(flights.isnull().sum())

We removed the obvoius categorical fields 'carrier', 'tailnum','flight','origin' and 'dest' - year and month could be candidates since they are not quanties. However, it's not clear that it will improve the model to convert them. 

It could be done using below code:   
`fight_withdummies = pd.get_dummies(flights, prefix='year_', columns=['year'])`


d) In the following, we are going to determine which factors cause departure time delays, and try to predict the length of these delays. However, for several departures, a *negative* delay have been reported. How do you interpret a negative delay? Consider if you want to modify the negative delays in some way. 

The negative delay is most likely a flight that arrives ahead of schedule. Two approches could be used.   

1. Consider the negative delays along side the delays - because if departures during rain are sometimes delayed and sometimes are ahead of schedule then it's not clear that rain is contributing to the delay. Whereas if you filter out the negatives incorrect patterns might emerge  

2. We could also choose to only look at delays and disregard flights that are ahead, since being ahead of schedule has a very limited positive impact compared to the negative of being delayed

We have chosen option 1

### Regression analysis: Predicting departure time delays

e) Extract the features and the target variable (in this case the departure time delays) from the dataframe. Split the dataset into test and train sets (technically, we ought to have done this before preprocessing. For the sake of simplicity, we do not conform to this best practice in this exercise).

In [ ]:
y = flights['dep_delay']

# We drop dep_delay from the dataset since it's now our taget - and we also drop arr_delay since that will not be known 
#  at the time we are trying to prodict departure delay.

# If we leave arr_delay in as a part of the dataset, prediction will be vastly improved - but that would not be 
#  representing a real-life senario

X = flights.drop(columns='dep_delay').drop(columns='arr_delay')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

y_train[:5]

f) Train a decision tree regressor for predicting departure time delays (you might want to experiment with a few different values of the hyperparameters to avoid too much overfitting). Plot the tree, and explain how decision trees can be used for regression analyses.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# max_depth=9 is the tipping poing of where we only get an increased accuracy on 
#   the trainingset (overfitting) by increasing the value. If we lower the value we get a lower
#   score on the test set (underfitting)
tree = DecisionTreeRegressor(max_depth=9)
tree.fit(X_train, y_train)

print("Accuracy on training data: {}".format(tree.score(X_train, y_train)))
print("Accuracy on testing data: {}".format(tree.score(X_test, y_test)))

In [ ]:
from sklearn.tree import export_graphviz
import graphviz as graphviz
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
dot_data = export_graphviz(tree, out_file=None,filled=True, rounded=True,special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

g) Do a regression analysis as the one above, but using a random forest instead of a single decision tree. Use a grid-search to determine a good set of hyperparameters. When you have found the best model, score your model on the test set. Comment on the result. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor


# max_depth=9 - since it seemed to produce good results in the prev. example
# max_features=5 - we want to find feature_importances_ so we reduce to a limited subset of our features

RandForReg = RandomForestRegressor(max_depth=9, random_state=0, n_estimators=60, max_features=5)
RandForReg.fit(X_train, y_train)

#RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
#           max_features='auto', max_leaf_nodes=None,
#           min_impurity_decrease=0.0, min_impurity_split=None,
#           min_samples_leaf=1, min_samples_split=2,
#           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
#           oob_score=False, random_state=0, verbose=0, warm_start=False)
print(RandForReg.score(X_test, y_test))
# Acc: ~0.29, random forest is doing only slightly better (~0.03) than the standard DecisionTreeRegressor






In [ ]:
print(RandForReg.feature_importances_)

n_features = X_train.columns
plt.barh(range(len(n_features)), RandForReg.feature_importances_, align='center')
plt.yticks(np.arange(n_features), n_features)
plt.xlabel("Feature importance")
plt.ylabel("Feature")

In [ ]:
X_train.columns


h) Plot the feature importances determined by the tree. Which feature is the most important? Do you have any idea as to why? Remove any features which cannot be used to predict departure time delays in any meaningful way, and redo the analysis. Comment on your results.

### Regression analysis: Predicting arrival time delays

In the last part of the exercise, we are going to try to predict arrival time delays as a function of departure time delays - it might be of interest to know how large a delay one should expect after the plane has departed from the airport. 

i) Train a decision tree or random forest regressor and an OLS (Ordinary least squares) to the dataset, and see how well arrival time delay. can be predicted based on departure time delay. 

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

flights = pd.read_csv('flights.csv')

# Set all null values to the mode - except in the target 'arr_delay'
for col in ['dep_time','dep_delay','arr_time','air_time','hour','minute']:
    flights[col] = flights[col].fillna(flights[col].mode()[0])    

# Only the target column now contains Null - drop all rows that don't have a valid target    
flights.dropna()

# The training target y is now "arr_delay" 
y = flights['arr_delay']

# We drop the taget "arr_delay" from the dataset and "air_time" since that is not known at the 
#  time we try to make the prediction
flights = flights.drop(columns='arr_delay').drop(columns='air_time') 

# We make dummy colums for the category features and drop the original column afterwards
for col in ['carrier','tailnum','flight','origin','dest']:
    flights = pd.get_dummies(flights, prefix=col, columns=[col])
    #flights = flights.drop(columns=col)


# We split the data
X_train, X_test, y_train, y_test = train_test_split(flights, y)
# Laptop   - 2min30
# i9-9900k - 26 sec


In [13]:
print(flights.isnull().sum())
#print(X_train[:10])

,Unnamed: 0,year,month,day,dep_time,dep_delay,arr_time,carrier,tailnum,flight,origin,dest,distance,hour,minute
314904,314905,2013,9,7,946.0,-4.0,1059.0,EV,N611QX,5463,LGA,BNA,764,9.0,46.0
280779,280780,2013,8,1,2227.0,88.0,2355.0,EV,N19966,3830,EWR,RIC,277,22.0,27.0
71886,71887,2013,11,18,1219.0,19.0,1514.0,AA,N329AA,3,JFK,LAX,2475,12.0,19.0
23410,23411,2013,1,28,627.0,-3.0,815.0,US,N564UW,1507,EWR,CLT,529,6.0,27.0
180538,180539,2013,4,17,954.0,-5.0,1212.0,US,N751UW,1277,LGA,CLT,544,9.0,54.0
302946,302947,2013,8,25,905.0,-10.0,1039.0,MQ,N544MQ,3565,LGA,CLT,544,9.0,5.0
59539,59540,2013,11,5,655.0,-10.0,1125.0,B6,N643JB,3,JFK,SJU,1598,6.0,55.0
99152,99153,2013,12,18,1012.0,6.0,1204.0,EV,N832AS,5736,LGA,IAD,229,10.0,12.0
150958,150959,2013,3,16,1857.0,0.0,2136.0,DL,N3738B,2159,JFK,MCO,944,18.0,57.0
71341,71342,2013,11,17,1954.0,22.0,2339.0,B6,N508JB,161,JFK,SMF,2521,19.0,54.0


In [ ]:
from sklearn.ensemble import RandomForestRegressor
RandForReg = RandomForestRegressor(max_depth=9, random_state=0, n_estimators=60, max_features=5)
RandForReg.fit(X_train, y_train)

print(RandForReg.score(X_test, y_test))


j) Plot the arrival time delays as a function of the departure time delay, and show the predictions from each of the two regressors.

k) Based on the results obtained above, make a plot that extrapolates a little bit in order to predict delays slightly larger than the largest delay found in the dataset. Which model do you think gives the most trustworthy extrapolation? 

l) Hopefully you found that it is possible to predict arrival time delays quite confidently from departure time delays. See if you can improve these predictions by including some (or all) of the other features. You are encouraged to try out several different machine learning algorithms.